Check if there is a consistent structure in the main fields of all json files:

In [12]:
import os, json
import pandas as pd

path_to_json = '../webscraping/exports/'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]

main_fields = ["sessionInfo", "mashrooh"]
sessionInfo_fields = [
    "term",
    "ejlasie",
    "preset_mps",
    "session",
    "date"]

max_orders = 0


print("By now, a session might have up to " + str(max_orders) + " number of orders.")

By now, a session might have up to 0 number of orders.


Transform session information JSONs to the dataframe session_df:

In [63]:
import pandas as pd

sessions_fields = {
    "term" : [],
    "ejlasie": [],
    "preset_mps": [],
    "session": [],
    "date": []}

session_df = pd.DataFrame(data=sessions_fields)

for json_file in json_files:
    # Opening JSON file
    json_file = open(path_to_json + json_file,'r') 

    # returns JSON object as a dictionary 
    string_data = json.load(json_file) 
    dict_data = json.loads(string_data)

    session_df.loc[session_df.shape[0]] = dict_data["sessionInfo"]

#extract year, month and day from the date string
session_df['date_year'] = session_df["date"].str.slice(0, 4)
session_df['date_month'] = session_df["date"].str.slice(5, 7)
session_df['date_day'] = session_df["date"].str.slice(8, 10)

session_df = session_df.sort_values(["date", "session"])
print(session_df.head(40))
session_df.to_csv("export_dataframes/session_information.csv")

   term ejlasie preset_mps session        date date_year date_month date_day
0    10       2        194     204  1397-02-19      1397         02       19
1    10       3        196     300  1397-12-19      1397         12       19
3    10       3        196     302  1397-12-21      1397         12       21
4    10       3        194     303  1397-12-26      1397         12       26
5    10       3        203     304  1398-01-18      1398         01       18
6    10       3        198     305  1398-01-19      1398         01       19
7    10       3        208     306  1398-01-20      1398         01       20
8    10       3        197     307  1398-01-20      1398         01       20
9    10       3        219     308  1398-01-25      1398         01       25
10   10       3        195     309  1398-01-26      1398         01       26
11   10       3        205     310  1398-01-27      1398         01       27
12   10       4        194     311  1398-01-27      1398         01       27

Extract orders from session information and save them in the dataframe order_df.

In [62]:
import re

order_fields = {
    "session_id": [],
    "order_id": [],
    "order_title": []}

order_df = pd.DataFrame(data=order_fields)

# Pattern to be used for removing the numbering html signs from order titles.
pattern = re.compile("^[0-9]*\r\n")

for json_file in json_files:
    # Opening JSON file
    json_file = open(path_to_json + json_file,'r') 

    # returns JSON object as a dictionary 
    string_data = json.load(json_file) 
    dict_data = json.loads(string_data)

    for order in dict_data["mashrooh"]:
        
        #Remove the numbering html marks from the order tilte.
        order_title = re.sub(pattern, '', dict_data["mashrooh"][order]["title"], count=0, flags=0)
        
        order_df.loc[order_df.shape[0]] = [
            dict_data["sessionInfo"]["session"],
            order[5:],
            order_title]

print(order_df.head(40))
order_df.to_csv("export_dataframes/session_orders.csv")


   session_id order_id                                        order_title
0         204        1                     اعلام رسميت جلسه و قرائت دستور
1         204        2                           تلاوت آياتي از قرآن مجيد
2         204        3  بيانات رئيس محترم مجلس شوراي اسلامي درخصوص خرو...
3         204        4  ارجاع طرح تشكيل وزارت ميراث فرهنگي، گردشگري و ...
4         204        5  ادامه رسيدگي به لايحه اصلاح قانون مبارزه با پو...
5         204        6  قرائت بيانيه نمايندگان مجلس شوراي اسلامي درخصو...
6         204        7  ناطقين جلسه آقايان: احمد همتي، سيدجواد حسيني‌ك...
7         204        8  تذكرات كتبي نمايندگان مجلس به مسئولان اجرايي كشور
8         204        9  تذكر آيين‌نامه‌اي و اخطار قانون اساسي نمايندگا...
9         204       10            اعلام ختم جلسه و تاريخ تشكيل جلسه آينده
10        300        1                     اعلام رسميت جلسه و قرائت دستور
11        300        2                           تلاوت آياتي از قرآن مجيد
12        300        3  بيانات رئيس مح